# GoodReads Recommendations Data Preparation

## Table of Contents

1. Background
2. Business Understanding
3. Data Understanding
4. Data Preparation

In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

## 1. Background

## 2. Business Understanding

## 3. Data Understanding

## 4. Data Preparation

Note that the three datasets are quite large, so they'll take some time to run.

In [13]:
df = pd.read_json('data/goodreads_books.json.gz', nrows =10000, compression='gzip', lines=True)[['isbn', 'language_code', 'average_rating', 'similar_books', 'description', 'link', 'authors', 'num_pages', 'isbn13', 'publication_year', 'url', 'book_id', 'ratings_count', 'title', 'title_without_series']]


OSError: [Errno 89] Operation canceled

In [12]:
# Trying to use chunking

# Define the columns you want to keep
cols = ['isbn', 'language_code', 'average_rating', 'similar_books', 'description', 'link', 'authors', 'num_pages', 'book_id', 'ratings_count', 'title']

# Initialize an empty DataFrame to store the data
gr_books = pd.DataFrame(columns=cols)

# Define the file name
file_name = 'data/goodreads_books.json.gz'

# Read the file in chunks
chunk_size = 100000  # Adjust the chunk size as needed
for chunk in pd.read_json(file_name, compression='gzip', lines=True, chunksize=chunk_size):
    chunk = chunk[cols]  # Keep only the specified columns
    gr_books = pd.concat([gr_books, chunk], ignore_index=True)

# Now you have the entire dataset stored in `gr_books`

OSError: [Errno 89] Operation canceled

In [7]:
gr_books = pd.read_json('data/goodreads_books.json.gz', compression='gzip', lines=True)

KeyboardInterrupt: 

In [6]:
gr_genres = pd.read_json('data/goodreads_book_genres_initial.json.gz', compression='gzip', lines=True)

OSError: [Errno 89] Operation canceled

In [4]:
# Loading in Data Sets
gr_reviews = pd.read_json('data/goodreads_reviews_spoiler.json.gz', compression='gzip', lines=True)
gr_books = pd.read_json('data/goodreads_books.json.gz', compression='gzip', lines=True)

OSError: [Errno 89] Operation canceled

### Users' Review Data

Here we want to end up with: rating, user id, book id, the review text for each user per book

In [8]:
gr_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378033 entries, 0 to 1378032
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   user_id           1378033 non-null  object        
 1   timestamp         1378033 non-null  datetime64[ns]
 2   review_sentences  1378033 non-null  object        
 3   rating            1378033 non-null  int64         
 4   has_spoiler       1378033 non-null  bool          
 5   book_id           1378033 non-null  int64         
 6   review_id         1378033 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(2), object(3)
memory usage: 64.4+ MB


In [9]:
gr_reviews.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,"[[0, This is a special book.], [0, It started ...",5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,"[[0, Recommended by Don Katz.], [0, Avail for ...",3,False,16981,a5d2c3628987712d0e05c4f90798eb67
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,"[[0, A fun, fast paced science fiction thrille...",3,True,28684704,2ede853b14dc4583f96cf5d120af636f
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,"[[0, Recommended reading to understand what is...",0,False,27161156,ced5675e55cd9d38a524743f5c40996e
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,"[[0, I really enjoyed this book, and there is ...",4,True,25884323,332732725863131279a8e345b63ac33e


In [11]:
gr_reviews['review_sentences'][2]

[[0, 'A fun, fast paced science fiction thriller.'],
 [0, "I read it in 2 nights and couldn't put it down."],
 [0,
  'The book is about the quantum theory of many worlds which states that all decisions we make throughout our lives basically create branches, and that each possible path through the decision tree can be thought of as a parallel world.'],
 [0,
  'And in this book, someone invents a way to switch between these worlds.'],
 [0, 'This was nicely alluded to/foreshadowed in this quote:'],
 [0, '"I think about all the choices we\'ve made that created this moment.'],
 [0, 'Us sitting here together at this beautiful table.'],
 [0,
  'Then I think of all the possible events that could have stopped this moment from ever happening, and it all feels, I don\'t know..." "What?"'],
 [0, '"So fragile."'],
 [0, 'Now he becomes thoughtful for a moment.'],
 [0,
  'He says finally, "It\'s terrifying when you consider that every thought we have, every choice we could possibly make, branches int

### Genre Data

We want to add this genre data to the books metadata

In [ ]:
gr_genres.info()

In [ ]:
gr_genres.head()

### Books Meta Data

We want one row per book

In [ ]:
gr_books.info()

In [ ]:
gr_books.head()